In [ ]:
file = "/home/thomas/Téléchargements/masa/SearchRequest2.xml"
file = "/home/thomas/Téléchargements/SearchRequest.xml"
file = "/home/thomas/Téléchargements/SearchRequest (1).xml"
file = "/home/thomas/Téléchargements/SearchRequest-10500.xml"

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
tree = ET.parse(file)
root = tree.getroot()

In [ ]:
root

In [ ]:
r = root[0]

In [ ]:
d = {}

In [ ]:
for c in r:
    if c.tag != "item":
        continue
    t = c.find("type").text
    if t in d:
        d[t] = d[t] + 1
    else:
        d[t] = 1

In [ ]:
c.find("type").text

In [ ]:
t

In [ ]:
t.text

In [ ]:
from grist_api import GristDocAPI

In [ ]:
SERVER = "https://grist.numerique.gouv.fr"
DOC_ID = "9mbWaZNUvym2"

In [ ]:
api = GristDocAPI(DOC_ID, server=SERVER)

In [ ]:
r = api.call("access")

In [ ]:
accesses = r.json()

In [ ]:
ac = {}
for a in accesses["users"]:
    aa = a["access"] or a["parentAccess"]
    if aa not in ac:
        ac[aa] = 1
    else:
        ac[aa] += 1
ac

In [ ]:
# drop viewers
#

In [ ]:
api = GristDocAPI(DOC_ID, server=SERVER)
accessResponse = api.call("access")
accesses = accessResponse.json()

viewersAccessToRemove = {
    person["email"]: None
    for person in accesses["users"]
    if (person["access"] or person["parentAccess"]) == "viewers"
}
editorAccesses = [
    person["email"] for person in accesses["users"] if person["access"] != "viewers"
]


people = [person for person in api.fetch_table("Personnes") if len(person.Email)]
toAdd = [p.Email for p in people if p.Email not in editorAccesses]

peopleEmails = [p.Email for p in people]
toRemove = [ea for ea in editorAccesses if ea not in peopleEmails]

updatesToDo = {
    **viewersAccessToRemove,
    **{email: None for email in toRemove},
    **{email: "editors" for email in toAdd},
}
if len(updatesToDo):
    data = {"delta": {"maxInheritedRole": "owners", "users": todo}}
    api.call("access", data, "PATCH")

    newAccessMap = {}
    for email in updatesToDo:
        access = updatesToDo[email]
        if access not in newAccessMap:
            newAccessMap[access] = []
        newAccessMap[access].append(email)
    newAccessMap

In [ ]:
updatesToDo = {
    "thomas@palya.fr": None,
    "thomas+edit@alya.fr": "editors",
    "t@t": "editors",
}

In [ ]:
newAccessMap = {}
for email in updatesToDo:
    access = updatesToDo[email]
    if access not in newAccessMap:
        newAccessMap[access] = []
    newAccessMap[access].append(email)
newAccessMap

In [ ]:
"Mise à jour des droits d'accès"

In [ ]:
[[a, "\n".join([f"- {e}" for e in newAccessMap[a]])] for a in newAccessMap]

In [ ]:
names = {None: "Suppression:", "editors": "Edition:"}

In [ ]:
body = "\n\n".join(
    ["\n".join([names[a], *[f"- {e}" for e in newAccessMap[a]]]) for a in newAccessMap]
)

In [ ]:
print(body)

In [ ]:
body.encode("ascii")

In [ ]:
body[32]